# Laboratory 1

### Group R. Rotoloni Gabriele n°733421

#### The input and output datasets can be found in the `DatasetsLocation.md` file in the `Datasets` folder.


After indicating the input and output paths, it's possibile to run the notebook altogether and read the results.
This notebook uses the datasets that have been produced by the notebook in the Preproc folder.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
activatedContractsInputPath = "C:\\Users\\rotol\\Downloads\\res\\Transformed_lavori_attivati.csv"
extendedContractsInputPath = "C:\\Users\\rotol\\Downloads\\res\\Transformed_lavori_prorogati.csv"

Create the dataset from the csv files

In [ ]:
activatedContracts = pd.read_csv(activatedContractsInputPath)
activatedContracts.head()

In [ ]:
extendedContracts = pd.read_csv(extendedContractsInputPath)
extendedContracts.head()

### Question: 
#### Which group of people (citizienship, age, sex, level of education) was the most successful at getting its own contract extended?

For this question I will create two dataframes having the education degree as index and multi-index columns for citizienship, sex and age.

The first one (extendedContractsByGroup) will indicate the number of extended contracts for each category. The second one (activatedExtendableContractsByGroup) represents the number of extendable contracts that have been activated by each category.

The result will be a dataframe having as values the division between the first and the second dataframe.

In [ ]:
#This is the dataframe for the number of extended contracts for each group of people. 

extendedContractsByGroup = extendedContracts.groupby(['TITOLOSTUDIO','ETA','GENERE','CITTADINANZA']).size()
for i in range(3):
    extendedContractsByGroup = extendedContractsByGroup.unstack()

extendedContractsByGroup = extendedContractsByGroup.fillna(0)
extendedContractsByGroup

In [ ]:
#This is the dataframe for the number of activated extendable contracts for each group of people. 

activatedExtendables = activatedContracts[activatedContracts.CONTRATTO == "PROROGABILE"]
activatedExtendableContractsByGroup = activatedExtendables.groupby(['TITOLOSTUDIO','ETA','GENERE','CITTADINANZA']).size()
for i in range(3):
    activatedExtendableContractsByGroup = activatedExtendableContractsByGroup.unstack()

activatedExtendableContractsByGroup = activatedExtendableContractsByGroup.fillna(0)
activatedExtendableContractsByGroup

To make the data more compact, I group up all the education degrees obtained after the High school diploma under one single category called "TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"

In [ ]:
#Sum the values of the rows together
advancedEdRowE = (extendedContractsByGroup.loc["DIPLOMA DI SPECIALIZZAZIONE"] + 
                  extendedContractsByGroup.loc["DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO"] + 
                  extendedContractsByGroup.loc["DIPLOMA UNIVERSITARIO"] + 
                  extendedContractsByGroup.loc["LAUREA - Vecchio o nuovo ordinamento"] + 
                  extendedContractsByGroup.loc["MASTER UNIVERSITARIO DI PRIMO LIVELLO"] + 
                  extendedContractsByGroup.loc["TITOLO DI DOTTORE DI RICERCA"] +
                  extendedContractsByGroup.loc["TITOLO DI STUDIO POST-LAUREA"])
#Add the row
extendedContractsByGroup.loc[len(extendedContractsByGroup)] = advancedEdRowE
extendedContractsByGroup.rename(index={12:"TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"}, inplace=True)
#Now i will delete the seven rows that have been combined
extendedContractsByGroup.drop(index=["DIPLOMA DI SPECIALIZZAZIONE","DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO",
                                     "DIPLOMA UNIVERSITARIO","LAUREA - Vecchio o nuovo ordinamento",
                                     "MASTER UNIVERSITARIO DI PRIMO LIVELLO","TITOLO DI DOTTORE DI RICERCA",
                                     "TITOLO DI STUDIO POST-LAUREA"], inplace=True)
extendedContractsByGroup

In [ ]:
#here I do the same thing for the activated Extendable Contracts
advancedEdRowA = (activatedExtendableContractsByGroup.loc["DIPLOMA DI SPECIALIZZAZIONE"] + 
                  activatedExtendableContractsByGroup.loc["DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO"] + 
                  activatedExtendableContractsByGroup.loc["DIPLOMA UNIVERSITARIO"] + 
                  activatedExtendableContractsByGroup.loc["LAUREA - Vecchio o nuovo ordinamento"] + 
                  activatedExtendableContractsByGroup.loc["MASTER UNIVERSITARIO DI PRIMO LIVELLO"] + 
                  activatedExtendableContractsByGroup.loc["TITOLO DI DOTTORE DI RICERCA"] +
                  activatedExtendableContractsByGroup.loc["TITOLO DI STUDIO POST-LAUREA"])
#Add the row
activatedExtendableContractsByGroup.loc[len(activatedExtendableContractsByGroup)] = advancedEdRowA
activatedExtendableContractsByGroup.rename(index={12:"TITOLO TERZIARIO DI SECONDO LIVELLO E DOTTORATO DI RICERCA"}, 
                                           inplace=True)
#Now i will delete the seven rows that have been combined
activatedExtendableContractsByGroup.drop(index=["DIPLOMA DI SPECIALIZZAZIONE","DIPLOMA TERZIARIO EXTRA-UNIVERSITARIO",
                                     "DIPLOMA UNIVERSITARIO","LAUREA - Vecchio o nuovo ordinamento",
                                     "MASTER UNIVERSITARIO DI PRIMO LIVELLO","TITOLO DI DOTTORE DI RICERCA",
                                     "TITOLO DI STUDIO POST-LAUREA"], inplace=True)
activatedExtendableContractsByGroup

The ratio is now computed

In [ ]:
successRatiodf = extendedContractsByGroup/activatedExtendableContractsByGroup
successRatiodf

In [ ]:
#the maxium value in this dataset:
successRatiodf.stack(level=[0,1,2]).index[np.argmax(successRatiodf.values)]

In [ ]:
analis = extendedContracts[(extendedContracts.TITOLOSTUDIO == 'LICENZA MEDIA')&
                  (extendedContracts.CITTADINANZA == 'STRANIERO')&
                  (extendedContracts.GENERE == 'F')&
                  (extendedContracts.ETA == "65 o piu'")]

analis.SETTOREECONOMICODETTAGLIO.unique().tolist()

The most "successful" group seems to be older foreigner female worker with a middle school degree. By watching at the types of job that were extended, on one hand, is probable that people in this group do not have interest in looking for a different job after the expiration of the contract, and on the other they are probably good enough for the employer, which is ok with renewing the contract if needed/possible.

Now we can compute other statistic:

- Average ratio for each education degree:

In [ ]:
extendedContractsByGroup.sum(axis=1)/activatedExtendableContractsByGroup.sum(axis=1)

It is interesting to see that the most "successful" categories are the middle school degree and the professional diploma. The middle school is probably related to what I wrote before and the professional diploma is probably due to the fact that one with this degree represents a specific professional figure.

Instead, the least "successful" are the highest education and those without a degree. The first category is probably low because people with high education should see a transformation into a permanent contract instead of a extension. The second category instead represents people that are easly replaceable.

- Average ratio per citizienship:

In [ ]:
extendedContractsByGroup.sum(axis=1, level=[0]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[0]).sum()

The citizienship status does not seem to play a big role on whether a contract will be extended or not

- Average ratio per citizienship and gender

In [ ]:
extendedContractsByGroup.sum(axis=1, level=[0,1]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[0,1]).sum()

For both Italians and foreigners, male got their contract extended more frequently

- Average ratio per age groups

In [ ]:
extendedContractsByGroup.sum(axis=1, level=[2]).sum()/activatedExtendableContractsByGroup.sum(axis=1, level=[2]).sum()

The average ratio is fairly consistent for all age groups except that people older than 65